## Installing and importing modules and libraries

In [138]:
!pip install bs4
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

## Webscrapping

In [139]:
# scrape webpage
toronto_postal_codes = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(toronto_postal_codes, 'html5lib') 

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Applying .shape method

In [140]:
df.shape

(103, 3)

## Dowloanding csv file with geographical coordinates

In [141]:
geo_data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Creating a new dataframe by merging the two previous dataframes

In [142]:
toronto_data = df.merge(geo_data, on='Postal Code')
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Importing new modules and libraries for maps and cluster analysis

In [143]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Libraries imported.


## Getting the geographical coordinates for Toronto

In [144]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


## Creating a map of Toronto's neighborhoods

In [145]:
# create map of Toronto using latitude and longitude values
map_toronto_neighborhoods = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighborhoods)  
    
map_toronto_neighborhoods

## Narrowing down the analysis
Since there are too many neighborhoods, I decided to narrow down the analysis to the borough of Downtwon Toronto, which is possibly the most busy area in terms of venues.

To start, I'm creating a new dataframe only with the neighboorhoods in Downtown Toronto

In [146]:
DT_data = toronto_data[toronto_data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DT_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


## Shaping method to the new dataframe

In [147]:
DT_data.shape

(17, 5)

## Getting the coordinates of Dowtown Toronto

In [148]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="downtown_toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dowtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dowtown Toronto are 43.6563221, -79.3809161.


## Creating a map of neighborhoods in Downtown Toronto

In [149]:
# create map of Dowtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(DT_data['Latitude'], DT_data['Longitude'], DT_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

## Defining Foursquare credentials to start exploring venues

In [150]:
{
    "tags": [
        "hide-input",
    ]
}
CLIENT_ID = 'LDR345YG0L5P213LREBTRYKQM0JY2KJR5KRORHMTOLX4XDJL' # your Foursquare ID
CLIENT_SECRET = 'GP4LH3S3LFILJKAQWQKFJ4HYTRB04EYIPGWH4YXHCXJY4Z5B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

## Getting the Top 100 venues in Downtown Toronto

In [151]:
latitude_DT = 43.6514990607
longitude_DT = -79.3834667995

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_DT, 
    longitude_DT, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LDR345YG0L5P213LREBTRYKQM0JY2KJR5KRORHMTOLX4XDJL&client_secret=GP4LH3S3LFILJKAQWQKFJ4HYTRB04EYIPGWH4YXHCXJY4Z5B&v=20180605&ll=43.6514990607,-79.3834667995&radius=500&limit=100'

## Send GET request to examine the results

In [152]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60b2810e7cb5d641c715710d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ad4c05ef964a520a6f620e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d164941735',
         'name': 'Plaza',
         'pluralName': 'Plazas',
         'primary': True,
         'shortName': 'Plaza'}],
       'id': '4ad4c05ef964a520a6f620e3',
       'location': {'address': '100 Queen St W',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Bay St',
        'distance': 85,
        'formattedAddress': ['100 Queen St W (at Bay St)',
         'Toronto ON M5H 2N1',
         'Canada'],
        'labeledLatLngs': [{'label': 'display

## Extract the categories of the venues

In [153]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Converting the results to a dataframe

In [154]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Nathan Phillips Square,Plaza,43.652270,-79.383516
1,Downtown Toronto,Neighborhood,43.653232,-79.385296
2,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
3,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156
4,John & Sons Oyster House,Seafood Restaurant,43.650656,-79.381613


In [155]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Getting nearby venues in the area of Downtown Toronto

In [156]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [157]:
downtown_toronto_venues = getNearbyVenues(names=DT_data['Neighborhood'],
                                   latitudes=DT_data['Latitude'],
                                   longitudes=DT_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [158]:
downtown_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


## Check venue category for each neighborhood in Downtown Toronto

In [159]:
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,62,62,62,62,62,62
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [160]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 203 uniques categories.


## Analyzing the neighborhoods and the frequency of venue categories

In [161]:
# one hot encoding
DT_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DT_toronto_onehot.columns[-1]] + list(DT_toronto_onehot.columns[:-1])
DT_toronto_onehot = DT_toronto_onehot[fixed_columns]

DT_toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,...,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [162]:
DT_toronto_onehot.shape

(1087, 203)

In [163]:
DT_toronto_grouped = DT_toronto_onehot.groupby('Neighborhood').mean().reset_index()
DT_toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,...,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.016949,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.016949,0.050847,0.000000,0.000000,0.00,0.016949,0.016949,0.000000,0.033898,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.00000,0.016949,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033898,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.033898,0.000000,0.016949,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.016949,0.00,0.016949,0.000000,0.00,0.000000,0.00,0.016949,0.000000,0.00,0.000000,0.016949,0.000000,0.000000,0.00,0.00,0.016949,0.000000,0.00000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.071429,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.071429,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.071429,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.000000
2,Central Bay Street,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.016129,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.032258,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.032258,0.032258,0.000000,...,0.000000,0.000000,0.016129,0.00,0.000000,0.032258,0.000000,0.048387,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.016129,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.016129,0.00,0.000000,0.000000,0.00,0.000000,0.032258,0.000000,0.000000,0.00,0.00,0.016129,0.000000,0.00000,0.016129
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.062500,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.000000
4,Church and Wellesley,0.025316,0.012658,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,

## Checking the most common venues in each neighborhood and putting the results into a dataframe

In [164]:
num_top_venues = 5

for hood in DT_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DT_toronto_grouped[DT_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.10
1  Cocktail Bar  0.07
2        Bakery  0.05
3      Beer Bar  0.03
4      Pharmacy  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3  Sculpture Garden  0.07
4   Harbor / Marina  0.07


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.18
1                 Café  0.06
2   Italian Restaurant  0.05
3       Sandwich Place  0.05
4  Japanese Restaurant  0.03


----Christie----
                venue  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12
3  Italian Restaurant  0.06
4         Candy Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.06
2  Japanese Restaurant  0.06
3           Restaurant  0.04
4        

In [165]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Checking the top 10 venues per neighborhood in Downtown Toronto

In [166]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DT_toronto_grouped['Neighborhood']

for ind in np.arange(DT_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Restaurant,Farmers Market,Seafood Restaurant,Cheese Shop,Pharmacy,Indian Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Lounge,Airport Service,Plane,Boutique,Rental Car Location,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bank,Bubble Tea Shop,Burger Joint,Thai Restaurant,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Candy Store,Restaurant,Baby Store,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Hotel,Yoga Studio,Men's Store,Pub


## Clustering neighborhoods in Downtown Toronto

In [167]:
# set number of clusters
kclusters = 4

DT_toronto_grouped_clustering = DT_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

## Putting the results into a dataframe

In [168]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

DT_toronto_merged = DT_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
DT_toronto_merged = DT_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DT_toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Restaurant,Breakfast Spot,Pub,Café,Theater,Mexican Restaurant,Beer Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Hotel,Fast Food Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Gastropub,Cocktail Bar,Creperie,Restaurant,Department Store,Cosmetics Shop,Clothing Store,Gym
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Restaurant,Farmers Market,Seafood Restaurant,Cheese Shop,Pharmacy,Indian Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bank,Bubble Tea Shop,Burger Joint,Thai Restaurant,Japanese Restaurant


## Visualizing the cluster results in a map

In [169]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DT_toronto_merged['Latitude'], DT_toronto_merged['Longitude'], DT_toronto_merged['Neighborhood'], DT_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters in Downtown Toronto

### Cluster 1 - Food venues
This represents the area in the inner circle of Dowtown Toronto. Food venues in all types, specially coffee shops tend to be higlhy concentrated in this area.


In [170]:
DT_toronto_merged.loc[DT_toronto_merged['Cluster Labels'] == 0, DT_toronto_merged.columns[[1] + list(range(4, DT_toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.360636,0,Coffee Shop,Bakery,Park,Restaurant,Breakfast Spot,Pub,Café,Theater,Mexican Restaurant,Beer Store
1,Downtown Toronto,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Hotel,Fast Food Restaurant
2,Downtown Toronto,-79.375418,0,Coffee Shop,Café,Gastropub,Cocktail Bar,Creperie,Restaurant,Department Store,Cosmetics Shop,Clothing Store,Gym
3,Downtown Toronto,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Restaurant,Farmers Market,Seafood Restaurant,Cheese Shop,Pharmacy,Indian Restaurant
4,Downtown Toronto,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bank,Bubble Tea Shop,Burger Joint,Thai Restaurant,Japanese Restaurant
6,Downtown Toronto,-79.384568,0,Coffee Shop,Café,Restaurant,Clothing Store,Gym,Deli / Bodega,Thai Restaurant,Hotel,Seafood Restaurant,Salad Place
7,Downtown Toronto,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Restaurant,Pizza Place,Scenic Lookout,Brewery,Italian Restaurant
8,Downtown Toronto,-79.381576,0,Coffee Shop,Hotel,Café,Salad Place,Japanese Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,Breakfast Spot,Gastropub
9,Downtown Toronto,-79.379817,0,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Gym,Seafood Restaurant,Cocktail Bar,Deli / Bodega,American Restaurant
10,Downtown Toronto,-79.400049,0,Café,Bookstore,Bar,Japanese Restaurant,Bakery,Yoga Studio,Italian Restaurant,Beer Store,Sandwich Place,Pub


### Cluster 2 - Leisure venues
As we move a little bit from the inner circle of Downtown Toronto, the type of venues start to change so the number of them. This cluster differentiates from the others by combining a bigger number of *leisure* venues.

In [171]:
DT_toronto_merged.loc[DT_toronto_merged['Cluster Labels'] == 1, DT_toronto_merged.columns[[1] + list(range(4, DT_toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,-79.377529,1,Park,Trail,Playground,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


### Cluster 3 - Residential area
With grocery store being the most common venue in this cluster, probably this is a more residential area comparing to the other clusters.

In [172]:
DT_toronto_merged.loc[DT_toronto_merged['Cluster Labels'] == 2, DT_toronto_merged.columns[[1] + list(range(4, DT_toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,-79.422564,2,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Candy Store,Restaurant,Baby Store,Coffee Shop


### Cluster 4 - Airport and travel
This cluster limits the surroundings of the airport, hence the most prevalent venues are some sort related to travel facilities.

In [173]:
DT_toronto_merged.loc[DT_toronto_merged['Cluster Labels'] == 3, DT_toronto_merged.columns[[1] + list(range(4, DT_toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,-79.39442,3,Airport Terminal,Airport Lounge,Airport Service,Plane,Boutique,Rental Car Location,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport
